<a href="https://colab.research.google.com/github/KarinaKatke/HiRiD/blob/main/getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import math
import numpy as np
import os
import pandas as pd
import re
import shutil
import sys

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/My Drive/HiRID/raw_stage'

In [4]:
# Extract Raw Data
import tarfile

file_path = '/content/drive/My Drive/HiRID/raw_stage/observation_tables_parquet.tar.gz'
extract_path = '/content/observation_tables'

with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

In [5]:
import os

extracted_files = os.listdir(extract_path)
print(extracted_files)

['observation_tables']


In [6]:
observation_table_path = os.path.join(extract_path, 'observation_tables')

In [7]:
# Listing of all parquet files
all_parquet_files = []
for root, dirs, files in os.walk(observation_table_path):
    for file in files:
        if file.endswith('.parquet'):
            all_parquet_files.append(os.path.join(root, file))

print(all_parquet_files)

['/content/observation_tables/observation_tables/parquet/part-78.parquet', '/content/observation_tables/observation_tables/parquet/part-12.parquet', '/content/observation_tables/observation_tables/parquet/part-64.parquet', '/content/observation_tables/observation_tables/parquet/part-113.parquet', '/content/observation_tables/observation_tables/parquet/part-122.parquet', '/content/observation_tables/observation_tables/parquet/part-140.parquet', '/content/observation_tables/observation_tables/parquet/part-84.parquet', '/content/observation_tables/observation_tables/parquet/part-138.parquet', '/content/observation_tables/observation_tables/parquet/part-218.parquet', '/content/observation_tables/observation_tables/parquet/part-161.parquet', '/content/observation_tables/observation_tables/parquet/part-119.parquet', '/content/observation_tables/observation_tables/parquet/part-123.parquet', '/content/observation_tables/observation_tables/parquet/part-90.parquet', '/content/observation_tables/

In [ ]:
#for parquet_file in all_parquet_files:
 #   df = pd.read_parquet(parquet_file)
  #  print(f"Reading file: {parquet_file}")
   # print(df.columns)

## Reading using Pandas
Required python packages are `pandas` and `pyarrow`

### Reading a part

In [8]:
part = 4

In [9]:
file_path = os.path.join(
    'observation_tables', 'observation_tables', 'parquet', f'part-{part}.parquet'
)

In [10]:
# Anzeigen der Spaltennmane des parquet-files

df_part = pd.read_parquet(file_path)
print(f"Datei geladen: {file_path}")
print(df_part.columns)


Datei geladen: observation_tables/observation_tables/parquet/part-4.parquet
Index(['datetime', 'entertime', 'patientid', 'status', 'stringvalue', 'type',
       'value', 'variableid'],
      dtype='object')


In [11]:
df_part['value'].count() #zählt wie viele Werte in der Spalte 'value' nicht leer sind, also wie viele Werte existieren

np.int64(2412421)

In [23]:
# die ersten 10 Zeilen anzeigen lassen des parquet files

print(df_part.head(10))

                 datetime               entertime  patientid  status  \
0 2111-04-07 17:50:00.000 2111-04-07 18:02:16.320         65       8   
1 2111-04-07 17:50:00.000 2111-04-07 18:02:16.333         65       8   
2 2111-04-07 18:00:00.000 2111-04-07 19:46:35.530         65       8   
3 2111-04-07 18:00:00.000 2111-04-07 19:46:36.140         65       8   
4 2111-04-07 18:02:54.220 2111-04-07 18:02:58.723         65       4   
5 2111-04-07 18:02:55.070 2111-04-07 18:02:58.803         65       4   
6 2111-04-07 18:03:28.100 2111-04-07 18:03:40.316         65      16   
7 2111-04-07 18:03:28.550 2111-04-07 18:03:40.363         65      16   
8 2111-04-07 18:04:34.850 2111-04-07 18:05:38.986         65      16   
9 2111-04-07 18:04:35.320 2111-04-07 18:05:39.033         65      16   

  stringvalue type  value  variableid  
0        None        90.0    10000400  
1        None       175.0    10000450  
2                     0.0    30005010  
3                     0.0    30005075  
4      

# Wie viele Einträge hat eine bestimmte Variable?

In [15]:
# spezifizieren welche value ausgezählt werden soll
variableid = 110
total_count = 0


In [16]:
# zählen wie viel data wir zu einer spezifischen value haben, in diesem parquet file

gefilterte_values = df_part[df_part['variableid'] == variableid]['value']
total_count = gefilterte_values.notna().sum()

print(f"\nGesamtanzahl nicht-NaN-Werte für VariableID {variableid}: {total_count}")
print(gefilterte_values)


Gesamtanzahl nicht-NaN-Werte für VariableID 110: 181348
35          0.0
38          0.0
45          0.0
49          0.0
53          0.0
           ... 
2412511    84.0
2412515    81.0
2412525    81.0
2412529    79.0
2412534    85.0
Name: value, Length: 181348, dtype: float32


In [24]:
# durch alle Parquet Files gehen und zählen:

# Zähler initialisieren
gesamt_count = 0

# Durchlaufen aller Parquet-Dateien
for file_path in all_parquet_files:
    print(f"Lade Datei: {file_path}")
    df = pd.read_parquet(file_path)

    # Filtern nach der gewünschten variableid
    gefilterte_values = df[df['variableid'] == variableid]['value']

    # Zählen nicht-NaN-Werte in der Spalte "value"
    count = gefilterte_values.notna().sum()
    gesamt_count += count

    print(f"{count} Werte in Datei gefunden.")

print(f"\n Gesamtanzahl nicht-NaN-Werte für VariableID {variableid}: {gesamt_count}")

Lade Datei: /content/observation_tables/observation_tables/parquet/part-78.parquet
214670 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-12.parquet
257442 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-64.parquet
196549 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-113.parquet
277988 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-122.parquet
263639 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-140.parquet
192614 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-84.parquet
194009 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-138.parquet
232272 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-218.